#### 0.思路说明如下：
1）把需要调整的参数制作为栅格（grid search，暂时作此翻译）；  
2）用R语言中的for循环依次调用栅格中的每一组参数训练模型，记录模型在model中，记录评估结果在perf中，然后删除对应模型，以免内存溢出；  
3）选择损失函数（loss function）最小的，即为最优模型；可以使用此模型的参数代入全部训练数据进行训练。

#### 1.加载测试程序需要的包

In [1]:
library(lightgbm)

Loading required package: R6


#### 2.加载测试数据（ lightgbm自带案例数据，请根据实际情况选择路径）

In [2]:
train <- read.table(file = 'C:/Users/Administrator/LightGBM/examples/multiclass_classification/multiclass.train') 
test <- read.table(file = 'C:/Users/Administrator/LightGBM/examples/multiclass_classification/multiclass.test')

In [3]:
dtrain <- lgb.Dataset(data.matrix(train[, 2:29]), label = train$V1, free_raw_data = FALSE)
dtest <- lgb.Dataset.create.valid(dataset = dtrain, data = data.matrix(test[, 2:29]), label = test$V1)
valids <- list(test = dtest)

#### 3.构建搜索栅格（也可以设置为随机搜索）

In [4]:
grid_search <- expand.grid(
    Learning_rate = seq(from = 0.1, to = 1, by = .1),
    Num_leaves = seq(from = 5, to = 50, by = 5)
)

#### 4.将建立的每一个模型放在model(一个list),交叉验证的评估放在perf中

In [6]:
perf <- numeric(nrow(grid_search))

#### 5.通过循环运行lightgbm（解决循环次数较多时会致使内存溢出的问题！）

In [7]:
for (i in 1:nrow(grid_search)) {
  model <- lgb.train(
    list(
      objective = "multiclass",
      metric = "multi_logloss",
      num_class = 5,
      #lambda_l1 = grid_search[i, "L1"],
      #lambda_l2 = grid_search[i, "L2"],
      learning_rate = grid_search[i, 'Learning_rate'],
      num_leaves = grid_search[i, 'Num_leaves']
    ),
    data = dtrain,
    nrounds = 10,
    valids = valids,
    early_stopping_rounds = 10,
    num_iterations = 100
  )
  perf[i] <- mean(unlist(model$record_evals$test$multi_logloss$eval))
  rm(model)
  gc() ## 清理R内存，此函数可不用，因为R在需要内存时会自动清理已经删除的文件（实际测试，偶尔不灵）
}

[1]:	test's multi_logloss:1.60654 
[2]:	test's multi_logloss:1.60331 
[3]:	test's multi_logloss:1.59858 
[4]:	test's multi_logloss:1.59524 
[5]:	test's multi_logloss:1.59264 
[6]:	test's multi_logloss:1.5897 
[7]:	test's multi_logloss:1.58666 
[8]:	test's multi_logloss:1.58356 
[9]:	test's multi_logloss:1.58114 
[10]:	test's multi_logloss:1.57751 
[1]:	test's multi_logloss:1.60398 
[2]:	test's multi_logloss:1.59552 
[3]:	test's multi_logloss:1.59224 
[4]:	test's multi_logloss:1.58658 
[5]:	test's multi_logloss:1.58019 
[6]:	test's multi_logloss:1.57503 
[7]:	test's multi_logloss:1.56855 
[8]:	test's multi_logloss:1.56191 
[9]:	test's multi_logloss:1.55838 
[10]:	test's multi_logloss:1.55316 
[1]:	test's multi_logloss:1.60175 
[2]:	test's multi_logloss:1.58929 
[3]:	test's multi_logloss:1.58078 
[4]:	test's multi_logloss:1.57315 
[5]:	test's multi_logloss:1.56593 
[6]:	test's multi_logloss:1.55781 
[7]:	test's multi_logloss:1.54811 
[8]:	test's multi_logloss:1.53976 
[9]:	test's multi_l

#### 6.结果表，第XX个模型的损失函数最小

In [8]:
cat("Model ", which.min(perf), " is lowest loss: ", min(perf), sep = "")

Model 77 is lowest loss: 1.400605

#### 7.损失函数最小的模型参数如下:

In [9]:
print(grid_search[which.min(perf), ])

   Learning_rate Num_leaves
77           0.7         40


#### 8.注：
1）本人测试以上程序所使用电脑为win10，两核四线程；  
2）本人所使用的lightgbm为CPU版本，未调用GPU；  
3) 本人所测试程序为R版本lightgbm，本人所用R为Microsoft R Server9.1.0版本；  
4）本人运行循环程序时长为50秒(删除model及清理内存致使比网络版多了5秒钟，即每迭代一次，多了0.05秒）；  
5）栅格设置处，learning_rate/num_leaves的每组值之间的间距设置不合理，例如learning的每个值之间的间距随着learning_rate的增加而增加更为合理，此处设置只是为了说明问题而已，请根据实际情况决定，勿采纳！